<a href="https://colab.research.google.com/github/Rutuja-Kolte/CodeBrewers/blob/master/GestureDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting Up**

In [ ]:
#To import everything required

from google.colab import files
import os

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import cv2
import pandas as pd

# To show images
from google.colab.patches import cv2_imshow

In [ ]:
# To mount drive and access files from it

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Used to create a symbolic link (symlink)

!ln -s "/content/drive/My Drive/CodeBrewers" "/content/CodeBrewers"

In [ ]:
# To load pre-trained model

model = keras.models.load_model('/content/CodeBrewers/GestureRecognition.h5')

**Function for Getting Image from Webcam**

In [ ]:
# To use a webcam to capture images for processing on the runtime

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true); // Resizes the output to fit the video element
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

**Predicting Hand Gestures Using the Model**

In [ ]:
from IPython.display import Image
gesture = ("down", "palm", "l", "fist", "fist_moved", "thumb", "index", "ok", "palm_moved", "c")
filename = take_photo()
img = cv2.imread('/content/photo.jpg')  # To get the picture taken from webcam
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Changes from BGR color space to gray
img = cv2.resize(img, (320, 120)) # To reduce image size
img = np.array(img)
img = np.reshape(img, (120,320,1))
# Modifies the data type, setting it to 32-bit floating point 
averageValue1 = np.float32(img)
while True:
  try:
    filename = take_photo()
    img = cv2.imread('/content/photo.jpg')  # Reads the picture taken from webcam
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Changes image color space from BGR to gray
    img = cv2.resize(img, (320, 120)) # Makes image smaller
    img = np.array(img)
    img = np.reshape(img, (120,320,1))
    cv2.accumulateWeighted(img, averageValue1, 0.02)  # Updates the running average
    resultingFrames1 = cv2.convertScaleAbs(averageValue1) # Converts the matrix elements to absolute values and converts the result to 8-bit
    resultingFrames1 = np.reshape(resultingFrames1, (120,320,1))
    cv2_imshow(img) # Original image
    cv2_imshow(resultingFrames1)  # Background using Running Average Method
    m = cv2.subtract(img,resultingFrames1)  # Foreground by subtracting background from original image
    m = np.reshape(m, (120,320,1))
    cv2_imshow(m)
    prediction = model.predict(np.expand_dims(m, axis = 0)) # Makes predictions
    ans = np.argmax(prediction[0])
    print(prediction[0][ans]) # Prints probability of prediction
    print(gesture[ans]) # Prints predicted gesture
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not grant the page permission to access it
    print(str(err))